# BlazingSQL Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

### Imports

In [3]:
import cudf
import numpy as np
from blazingsql import BlazingContext

### Sample DataFrame

In [14]:
df = cudf.DataFrame(
    [
          (39, 6.88, np.datetime64('2020-10-08T12:12:01'), np.timedelta64(14378,'s'), 'C', 'D', 'data'
            , 'RAPIDS.ai is a suite of open-source libraries that allow you to run your end to end data science and analytics pipelines on GPUs.')
        , (11, 4.21, None,                                 None                     , 'A', 'D', 'cuDF'
            , 'cuDF is a Python GPU DataFrame (built on the Apache Arrow columnar memory format)')
        , (31, 4.71, np.datetime64('2020-10-10T09:26:43'), np.timedelta64(12909,'s'), 'U', 'D', 'memory'
            , 'cuDF allows for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API.')
        , (40, 0.93, np.datetime64('2020-10-11T17:10:00'), np.timedelta64(10466,'s'), 'P', 'B', 'tabular'
            , '''If your workflow is fast enough on a single GPU or your data comfortably fits in memory on 
                 a single GPU, you would want to use cuDF.''')
        , (33, 9.26, np.datetime64('2020-10-15T10:58:02'), np.timedelta64(35558,'s'), 'O', 'D', 'parallel'
            , '''If you want to distribute your workflow across multiple GPUs or have more data than you can fit 
                 in memory on a single GPU you would want to use Dask-cuDF''')
        , (42, 4.21, np.datetime64('2020-10-01T10:02:23'), np.timedelta64(20480,'s'), 'U', 'C', 'GPUs'
            , 'BlazingSQL provides a high-performance distributed SQL engine in Python')
        , (36, 3.01, np.datetime64('2020-09-30T14:36:26'), np.timedelta64(24409,'s'), 'T', 'D', None
            , 'BlazingSQL is built on the RAPIDS GPU data science ecosystem')
        , (38, 6.44, np.datetime64('2020-10-10T08:34:36'), np.timedelta64(90171,'s'), 'X', 'B', 'csv'
            , 'BlazingSQL lets you ETL raw data directly into GPU memory as a GPU DataFrame (GDF)')
        , (17, 5.28, np.datetime64('2020-10-09T08:34:40'), np.timedelta64(30532,'s'), 'P', 'D', 'dataframes'
            , 'Dask is a flexible library for parallel computing in Python')
        , (10, 8.28, np.datetime64('2020-10-03T03:31:21'), np.timedelta64(23552,'s'), 'W', 'B', 'python'
            , None)
    ]
    , columns = ['num', 'float', 'datetime', 'timedelta', 'char', 'category', 'word', 'string']
)

# BlazingContext

#### blazignsql.BlazingContext()

In [7]:
bc = BlazingContext()

BlazingContext ready


In [11]:
bc = BlazingContext(
    dask_client=None
    , pool=True
    , initial_pool_size=4*(1024**3) ## 4 GB
    , network_interface='eth0'
)

BlazingContext ready


#### blazignsql.BlazingContext.s3()

In [12]:
_ = bc.s3(
    'bsql'
    , bucket_name = 'bsql'
)

(True,
 '',
 OrderedDict([('type', 's3'),
              ('bucket_name', 'bsql'),
              ('access_key_id', ''),
              ('secret_key', ''),
              ('session_token', ''),
              ('encryption_type', <S3EncryptionType.NONE: 1>),
              ('kms_key_amazon_resource_name', ''),
              ('endpoint_override', ''),
              ('region', '')]))

In [ ]:
_ = bc.s3(
    '<dir_name>'
    , bucket_name='<bucket_name>'
    , access_key_id='<ACCESS_KEY>'
    , secret_key='<SECRET_KEY>'
    , region="<region>"
    , endpoint_override="https://s3.filebase.com"
)

#### blazignsql.BlazingContext.gs()

In [ ]:
bc.gs('dir_name', 
      project_id='<project_id>', 
      bucket_name='<bucket_name>', 
      use_default_adc_json_file=False, 
      adc_json_file='<../file_path_to/credentials.json>')

#### blazignsql.BlazingContext.create_table()

In [15]:
bc.create_table('df', df)

In [23]:
bc.create_table('apache', '../data/apache_sample_head.parquet')

In [17]:
bc.create_table('apache', '../data/apache_sample_head.csv')

In [28]:
bc.create_table('parking_locations','s3://bsql/data/seattle_parking/parking_locations.parquet/')

#### blazignsql.BlazingContext.drop_table()

In [29]:
bc.drop_table('apache')

#### blazignsql.BlazingContext.list_tables()

In [21]:
bc.list_tables()

['df']

#### blazignsql.BlazingContext.describe_table()

In [30]:
bc.describe_table('parking_locations')

{'SourceElementKey': 'int64',
 'BlockfaceName': 'str',
 'SideOfStreet': 'str',
 'ParkingTimeLimitCategory': 'int64',
 'ParkingSpaceCount': 'int64',
 'PaidParkingArea': 'str',
 'PaidParkingSubArea': 'str',
 'ParkingCategory': 'str',
 'Location': 'str'}

#### blazignsql.BlazingContext.sql()

In [31]:
bc.sql('SELECT * FROM parking_locations LIMIT 3')

,SourceElementKey,BlockfaceName,SideOfStreet,ParkingTimeLimitCategory,ParkingSpaceCount,PaidParkingArea,PaidParkingSubArea,ParkingCategory,Location
0,1009,1ST AVE BETWEEN MADISON ST AND SPRING ST,SW,30,5,Commercial Core,Waterfront,Paid Parking,POINT (-122.3366575 47.60501765)
1,1018,1ST AVE BETWEEN STEWART ST AND VIRGINIA ST,NE,120,12,Belltown,South,Paid Parking,POINT (-122.34188878 47.61073498)
2,1045,1ST AVE N BETWEEN REPUBLICAN ST AND MERCER ST,W,120,11,Uptown,Core,Paid Parking,POINT (-122.35549857 47.62391565)


#### blazignsql.BlazingContext.explain()

In [32]:
bc.explain('SELECT * FROM parking_locations LIMIT 3')

'LogicalSort(fetch=[3])\n  LogicalTableScan(table=[[main, parking_locations]])\n'

#### blazignsql.BlazingContext.log()

In [36]:
bc.log("SELECT log_time, query_id, duration FROM bsql_logs WHERE info = 'Query Execution Done' ORDER BY log_time DESC")

,log_time,query_id,duration
0,2020-12-05 00:07:12.421,984749867,781.0
1,2020-12-05 00:07:07.608,1292265483,1156.0
2,2020-12-05 00:05:05.425,1168405988,1228.0


SQL
#### SELECT#### Yes
#### DISTINCT#### Yes
#### WHERE#### Yes
#### GROUP BY#### Yes
#### ORDER BY#### Yes
#### CASE#### Yes
#### LIKE#### Yes
#### CAST#### Yes

SQL Joins
#### INNER
#### FULL OUTER
#### LEFT OUTER
#### CROSS JOIN

SQL Unary Functions
#### FLOOR
#### CEILING
#### SIN
#### COS
#### ASIN
#### ACOS
#### TAN
#### ATAN
#### SQRT
#### COTAN
#### ABS
#### NOT
#### LN
#### LOG
#### RAND
#### ROUND
#### STDDEV
#### STDDEV_POP
#### STDDEV_SAMP
#### VARIANCE
#### VAR_SAMP
#### VAR_POP

SQL Binary Functions
#### Addition
#### Subtraction
#### Multiplication
#### Division
#### MOD
#### POWER
#### EQUAL TO
#### NOT EQUAL TO
#### LESS THAN
#### GREATER THAN
#### LESS THAN OR EQUAL TO
#### GREATER THAN OR EQUAL TO
#### COALESCE

SQL DateTime Functions
#### YEAR
#### MONTH
#### DAYOFMONTH
#### HOUR
#### MINUTE
#### SECOND
#### DAYOFWEEK
#### TO_DATE
#### TO_TIMESTAMP

SQL String Functions
#### CONCAT
#### SUBSTRING
#### CHAR_LENGTH
#### LEFT
#### RIGHT
#### LTRIM
#### RTRIM
#### REPLACE
#### UPPER
#### LOWER
#### REVERSE